In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import pathlib
from kerastuner.engine.hypermodel import HyperModel
from kerastuner.engine.hyperparameters import HyperParameters
from kerastuner.tuners.randomsearch import RandomSearch
print(tf.__version__)

In [ ]:
train_data_dir = "D://Betastop_dataset/insect_dataset/_train"
train_data_dir = pathlib.Path(train_data_dir)
image_count = len(list(train_data_dir.glob('*/*.jpg')) + list(train_data_dir.glob('*/*.jpeg')))
print(image_count)

validation_data_dir = "D://Betastop_dataset/insect_dataset/_validation"
validation_data_dir = pathlib.Path(validation_data_dir)

test_data_dir = "D://Betastop_dataset/insect_dataset/_test"
test_data_dir = pathlib.Path(test_data_dir)

In [ ]:
train_list_ds = tf.data.Dataset.list_files(str(train_data_dir/'*/*'))
for f in train_list_ds.take(5):
    print(f.numpy())
validation_list_ds = tf.data.Dataset.list_files(str(validation_data_dir/'*/*'))
test_list_ds = tf.data.Dataset.list_files(str(test_data_dir/'*/*'))


In [ ]:
CLASS_NAMES = np.array([item.name for item in train_data_dir.glob('*')])
import os
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == CLASS_NAMES

print(str(CLASS_NAMES))


In [ ]:
IMG_HEIGHT = 224
IMG_WIDTH = 224
def decode_img(img):
  # convert the compressed string to a 3D uint8 tensor
  img = tf.image.decode_jpeg(img, channels=3)
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  img = tf.image.convert_image_dtype(img, tf.float32)
  # resize the image to the desired size.
  return tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
def process_path(file_path):
  label = get_label(file_path)
  # load the raw data from the file as a string
  img = tf.io.read_file(file_path)
  img = decode_img(img)
  print(label[0])
  return img, label
train_labeled_ds = train_list_ds.map(process_path, num_parallel_calls = 8)
validation_labeled_ds = validation_list_ds.map(process_path, num_parallel_calls = 8)
test_labeled_ds = test_list_ds.map(process_path, num_parallel_calls=8)

In [ ]:
for image, label in train_labeled_ds.take(1):
  print("Image shape: ", image.numpy().shape)
  print("Label: ", label.numpy())

test_label_list = []
for image, label in test_labeled_ds.take(-1):
    test_label_list.append(label.numpy())

In [ ]:
BATCH_SIZE = 32
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds


In [ ]:

train_ds = prepare_for_training(train_labeled_ds, cache = './train.tfcache')


validation_ds = prepare_for_training(validation_labeled_ds, cache = './val.tfcache')

test_ds = prepare_for_training(test_labeled_ds, cache = './test.tfcache')

print(test_ds)

In [ ]:
from progress.bar import Bar
label_list = []

alpha, beta = next(iter(test_ds))
a = beta.numpy()

a = a.astype(int)
print(a)
print(a[5])



In [ ]:
label_list = []
for image, label in test_ds.take(152):
    label_list.append(label.numpy())
    

In [ ]:
from focal_loss import SparseCategoricalFocalLoss
import tensorflow_addons as tfa


model = keras.Sequential()
model.add(hub.KerasLayer("https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1",
                        trainable = False))
model.add(keras.layers.Dropout(rate=0.3))
model.add(keras.layers.Dense(40, activation = "softmax"))
model.build([None, 224, 224, 3])


model.compile(optimizer=keras.optimizers.Adam(lr=0.001),
              loss = keras.losses.categorical_crossentropy,
              metrics = ['accuracy', tfa.metrics.CohenKappa(num_classes=40), tfa.metrics.F1Score(num_classes=40), keras.metrics.TrueNegatives(), keras.metrics.TruePositives(),
                         keras.metrics.FalseNegatives(), keras.metrics.FalsePositives()])


In [ ]:
#history = model.fit(train_ds,
          #steps_per_epoch=np.round(image_count/BATCH_SIZE) - 1,
          #epochs = 20,
          #validation_data= validation_ds,
          #validation_steps= 30)

predicted_list = model.predict(test_ds, batch_size=32, steps=152, verbose = 1)

print(predicted_list.shape)

In [ ]:

print(predicted_list)

In [ ]:
test_label_list = np.asarray(test_label_list)
print(test_label_list.shape)

In [ ]:
import sklearn
label_list = np.asarray(label_list)
label_list = label_list.astype(int)
label_list = label_list.reshape((4864, 40))
print(label_list[1340])
print(predicted_list[1340])

hello = sklearn.metrics.roc_auc_score(label_list, predicted_list, average='micro', multi_class='ovr')

In [ ]:
print(hello)


In [ ]:
model.evaluate(test_ds, steps=152)

